<a href="https://colab.research.google.com/github/altdeep/causalML/blob/master/book/chapter%206/Monte_Hall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.4 MB/s 


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination

monty_hall_model = BayesianNetwork([    #A
    ('Host Inclination', 'Host Door Selection'),    #A
    ('Door with Car', 'Host Door Selection'),    #A
    ('Player First Choice', 'Host Door Selection'),    #A
    ('Player First Choice', 'Player Second Choice'),    #A
    ('Host Door Selection', 'Player Second Choice'),    #A
    ('Strategy', 'Player Second Choice'),    #A
    ('Player Second Choice', 'Win or Lose'),    #A
    ('Door with Car', 'Win or Lose')    #A
])    #A

p_host_inclination = TabularCPD(       #B
    variable='Host Inclination',    #B
    variable_card=2,    #B
    values=[[.5], [.5]],    #B
    state_names={'Host Inclination': ['left', 'right']}    #B
)    #B

p_door_with_car = TabularCPD(    #C
    variable='Door with Car',    #C
    variable_card=3,    #C
    values=[[1/3], [1/3], [1/3]],    #C
    state_names={'Door with Car': ['1st', '2nd', '3rd']}    #C
)    #C

p_player_first_choice = TabularCPD(    #D
    variable='Player First Choice',    #D
    variable_card=3,    #D
    values=[[1/3], [1/3], [1/3]],    #D
    state_names={'Player First Choice': ['1st', '2nd', '3rd']}    #D
)    #D

p_host_strategy = TabularCPD(    #E
    variable='Strategy',    #E
    variable_card=2,    #E
    values=[[.5], [.5]],    #E
    state_names={'Strategy': ['stay', 'switch']}    #E
)    #E

f_host_door_selection = TabularCPD(    #F
    variable='Host Door Selection',    #F
    variable_card=3,    #F
    values=[    #F
        [0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0],    #F
        [1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1],    #F
        [0,1,0,1,0,0,0,0,0,1,1,0,1,1,0,0,0,0]    #F
    ],    #F
    evidence=['Host Inclination', 'Door with Car', 'Player First Choice'],    #F
    evidence_card=[2, 3, 3],    #F
    state_names={    #F
        'Host Door Selection':['1st', '2nd', '3rd'],    #F
        'Host Inclination': ['left', 'right'],    #F
        'Door with Car': ['1st', '2nd', '3rd'],    #F
        'Player First Choice': ['1st', '2nd', '3rd']    #F
    }    #F
)    #F

f_second_choice = TabularCPD(    #G
    variable='Player Second Choice',    #G
    variable_card=3,    #G
    values=[    #G
        [1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0],    #G
        [0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1],    #G
        [0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0]    #G
    ],    #G
    evidence=['Strategy', 'Host Door Selection', 'Player First Choice'],    #G
    evidence_card=[2, 3, 3],    #G
    state_names={    #G
        'Player Second Choice': ['1st', '2nd', '3rd'],    #G
        'Strategy': ['stay', 'switch'],    #G
        'Host Door Selection': ['1st', '2nd', '3rd'],    #G
        'Player First Choice': ['1st', '2nd', '3rd']    #G
    }    #G
)    #G

f_win_or_lose = TabularCPD(    #H
    variable='Win or Lose',    #H
    variable_card=2,    #H
    values=[    #H
        [1,0,0,0,1,0,0,0,1],    #H
        [0,1,1,1,0,1,1,1,0],    #H
    ],    #H
    evidence=['Player Second Choice', 'Door with Car'],    #H
    evidence_card=[3, 3],    #H
    state_names={    #H
        'Win or Lose': ['win', 'lose'],    #H
        'Player Second Choice': ['1st', '2nd', '3rd'],    #H
        'Door with Car': ['1st', '2nd', '3rd']    #H
    }    #H
)    #H

monty_hall_model.add_cpds(    #I
    p_host_inclination,    #I
    p_door_with_car,    #I
    p_player_first_choice,    #I
    p_host_strategy,    #I
    f_host_door_selection,    #I
    f_second_choice,    #I
    f_win_or_lose    #I
)    #I

#A Build the causal DAG.
#B A CPD for the variable Host Inclination.  In cases when the player chooses the door with the car, the host has a choice between the two other doors. This variable is "left" when the host is inclined to choose the left-most door, and "right" if the host is inclined to choose the right-most door.
#C A CPD for the variable representing which door has the prize card.  Assume each door has equal probability of having the car.
#D A CPD for variable representing the player's first door choice.  Each door has equal probability of being chosen.
#E A CPD for variable representing the player's strategy.  "Stay" is the strategy of staying with the first choice, and "switch" is the strategy of switching doors.
#F Structural assignment function for the host's door selection.
#G Structural assignment function for the player's second choice.
#H Structural assignment function for whether the player wins or loses.
#I Add the CPDs to the causal DAG.

In [ ]:
inference_engine = VariableElimination(monty_hall_model)
print(inference_engine.query(['Win or Lose'], evidence={'Strategy': 'stay'}))
print(inference_engine.query(['Win or Lose'], evidence={'Strategy': 'switch'}))

+-------------------+--------------------+
| Win or Lose       |   phi(Win or Lose) |
+===================+====================+
| Win or Lose(win)  |             0.3333 |
+-------------------+--------------------+
| Win or Lose(lose) |             0.6667 |
+-------------------+--------------------+
+-------------------+--------------------+
| Win or Lose       |   phi(Win or Lose) |
+===================+====================+
| Win or Lose(win)  |             0.6667 |
+-------------------+--------------------+
| Win or Lose(lose) |             0.3333 |
+-------------------+--------------------+


In [ ]:
print(inference_engine.query(['Strategy'], evidence={'Win or Lose': 'win'}))

+------------------+-----------------+
| Strategy         |   phi(Strategy) |
+==================+=================+
| Strategy(stay)   |          0.3333 |
+------------------+-----------------+
| Strategy(switch) |          0.6667 |
+------------------+-----------------+
